*TP réalisé par Rachel Blin dans le cadre du cours d'Alexandrina Rogozan*

# Codage cyclique

Pour ce TP, on considère un code correcteur d'erreur cyclique par blocs ayant ayant des mots-code de longueur 7 bits dans lesquels on retrouve m=3 bits de parité. Les n=7 bits constituant un mot-code sont considérés comme coefficients d'un polynôme de degré n-1 soit :  

c(x) = d0 + d1x + d2x² + ... + d6x^6    

Le polynôme générateur utilisé pour obtenir un mot-code est le suivant : 

g(x) = x^3 + x + 1

Afin de retrouver tous les mots-code constituant ce code cyclique il suffit de multiplier les k bits d'info avec k = n-m par le polynôme générateur.

## Ecriture des mots-code 

1) Dans un premier temps, déterminer toutes les valeurs possibles des bits d'info

In [59]:
import numpy as np
import itertools

n = 7
m = 3

def values_bits_info(m, n):
    """ Function that returns all the possibles values for the information bits
    
    # Arguments:
        - m: The number of parity bits
        - n: The length of the codeword
    
    # Returns:
        A list of all the possible combinations of the information bits    
    """
    k = n-m
    binary_list = [0,1]
    res = [list(reversed(list(x))) for x in itertools.product(binary_list,  repeat =k)]
    
    return res


bits_info = values_bits_info(m,n)
bits_info

[[0, 0, 0, 0],
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [1, 1, 0, 0],
 [0, 0, 1, 0],
 [1, 0, 1, 0],
 [0, 1, 1, 0],
 [1, 1, 1, 0],
 [0, 0, 0, 1],
 [1, 0, 0, 1],
 [0, 1, 0, 1],
 [1, 1, 0, 1],
 [0, 0, 1, 1],
 [1, 0, 1, 1],
 [0, 1, 1, 1],
 [1, 1, 1, 1]]

2) Ecrivez maintenant tous les mots-code de ce code cyclique.

In [65]:
from numpy.polynomial import polynomial 

g = np.array([1, 1, 0, 1])

def polynomial_multiplication(x, y):
    """ A function to multiply two binary polynoms
    
    # Arguments:
        - x: A binary polynom
        - y: A binary polynom
        
    # Returns:
        A binary polynom
    """
    
    #On initialise le vecteur du résultat et le polynomial calculé 
    res = np.zeros(len(x)+len(y)-1)
    res = res.tolist()
    poly = polynomial.polymul(x,y)
    
    #On enregistre le polynomial dans notre résultat, pour avoir la
    #bonne dimensio, 
    for i in range(len(poly)):
        res[i] = poly[i]
  
    #On vérifie et corrige si un nombre est supérieur à 1   
    for i in range(len(res)) : 
        if (res[i]%2 == 0) : 
            res[i] = 0
        if (res[i]%2 == 1) : 
            res[i] = 1
    return res



def cyclic_codeword(bits_info, g):
    """ A function that computes all the possible cyclic codewords
    
    # Arguments:
        - bits_info: A list containing all the possibles information bits combination
        - g: The generator polynom
    
    """
    res = list()
    
    for x in bits_info :
        res.append(polynomial_multiplication(g,x))
    
    return res

codeword = cyclic_codeword(values_bits_info(m,n),g)
print('Mot code du code cyclique :')
for i in range(len(codeword)):
    print(codeword[i])
print()


Mot code du code cyclique :
[0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0]
[0, 1, 1, 0, 1, 0, 0]
[1, 0, 1, 1, 1, 0, 0]
[0, 0, 1, 1, 0, 1, 0]
[1, 1, 1, 0, 0, 1, 0]
[0, 1, 0, 1, 1, 1, 0]
[1, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 0, 1]
[1, 1, 0, 0, 1, 0, 1]
[0, 1, 1, 1, 0, 0, 1]
[1, 0, 1, 0, 0, 0, 1]
[0, 0, 1, 0, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 1, 1]
[1, 0, 0, 1, 0, 1, 1]



On dit qu'un code est cyclique parce que les mots-code de celui-ci possèdent un motif récurrent (sans compter les mots-code dont tous les bits sont à 0 ou à 1) et qui peut être inversé. 

3) Quel est le motif de ce code cyclique?

Le motif est 1101 

## Calcul de la distance minimale de ce code

Pour rappel, la distance de Hamming entre deux séquences de même longueur est le nombre de caractères qui différent entre ces deux séquences.

__Exemple__ :  
La distance de Hamming entre "*1000101*" et "*1100101*" est de 1.  
La distance de Hamming entre "*1000101*" et "*0000111*" est de 2.  

4) Calculez la distance minimale de Hamming pour ce code. Pour celà, il est plus judicieux de calculer la distance de Hamming entre le mot-code dont tous les bits sont à zéros et les autres mots_codes et de retenir la distance minimale.

In [66]:
def Hamming_distance(codewords):
    """ This function computes the minimal Hamming thistance for the codewords
    
    # Argument:
        - codewords: A list containing all the codewords
        
    # Returns:
        The minimal Hamming distance of these codewords
    """
    #On initialise notre vecteur et la distance minimale. 
    reference = np.zeros(len(codewords[0]))
    dist_min = len(codewords[0]) 
    
    
    
    #On fait la distance de Hamming pour chaque mot code
    for item in codewords :
        dist_temp = 0
        for i in range(len(codewords[0])):
            if (item[i] != reference[i]) :
                dist_temp += 1 
        
    #Si la distance du mot code avec 0000000 est plus petite que celle
    #retenue, on l'enregistre. On vérifie cependant que ce n'est pas la
    #distance entre 0000000 et 0000000. 
        if ((dist_temp < dist_min) & (dist_temp != 0)) : 
            dist_min = dist_temp 
            
            
    return dist_min

Hamming_distance(cyclic_codeword(values_bits_info(m,n),g))

3

Le nombre maximal d'erreurs corrigibles par ce code est déterminé par le plus grand entier strictement inférieur à la distance minimale de Hamming / 2.  

5) calculez le nombre maximal d'erreurs corrigibles par ce code

In [67]:
def number_errors(d_Hamming):
    """ A function that computes the number of errors that this cyclic code can correct
    
    # Argument:
        - d_Hamming: The minimal Hamming distance
        
    # Returns:
        The number of errors this code can correct
    """
    return int(d_Hamming/2)

## Calcul des mots-code sous forme systématique 

Pour rappel, dans notre cas de figure on a m = 3 bits de parité.    

Le polynôme des bits de parité (p(x)) est le reste de la division du polynome formé à partir des bits d'information (i(x)) multiplié par x^m par g(x) soit :  
p(x) = reste((i(x) * x^m) / g(x))

Ces bits de parité permettent d'écrire les mots-code sous forme systématique avec d0, d1 et d2 les coefficients du polynôme des bits de parité p(x) et d3, d4, d5 et d6 les coefficents du polynôme d'information i(x).    

6) Ecrivez les mots-code sous forme systématique.

In [68]:
def polynomial_soustraction(x, y):
    """ A function that computes the binary polynomial soustraction of two polynoms
    
    # Arguments:
        - x: A binary polynom
        - y: A binary polynom
        
    # Returns:
        A binary polynom   
    """
    
    deg_max = max(len(x),len(y)) 
    
    resultat = np.zeros(deg_max)
    resultat = resultat.tolist()
    
    for i in range (len(x)) :
        resultat[i] = x[i]
    for j in range (len(y)):
        resultat[j] = resultat[j] - y[j]
    
    for l in range (deg_max):
        if (resultat[l]<0) : #car 0-1 = -1
            resultat[l] = 1
    return resultat

    
    
    
def rest_division(x, y):
    """ A function that computes the rest of the binary division
    
    # Arguments:
        - x: A binary polynom
        - y: A binary polynom
        
    # Returns:
        A binary polynom
    """
    pol_x = 0; 
    pol_y = 0;
    for i in range (len(x)):
        if (x[i]==1): #dernière position du 1 dans x
            pol_x = i
    for j in range(len(y)): #dernière position du 1 dans y
        if (y[j]==1):
            pol_y = j      
    while pol_x>=pol_y :
        pol_diff = pol_x - pol_y #quotient
        y_temp = polynomial_multiplication(y, poly_x_m(pol_diff))
        x = polynomial_soustraction(x, y_temp) #le reste
        pol_x = 0 #comme on a modifié x, on réinitialise g_x
        for i in range (len(x)):
            if (x[i]==1):
                pol_x = i
    return x

def poly_x_m(m):
    """ A function that creates the polynom x^m
    
    # Arguments:
        - m: The degree of the desired polynom
    
    # Returns:
        A binary polynom
    """
    res = np.zeros(m+1)
    res = res.tolist()
    res[m] = 1
    
    return res


def bits_info_multi_x_m(bits_info, x):
    """ A function that multiplies the polynom corresponding to the information bits with the polynom x^m
    
    # Arguments:
        - bits_info: A list containing all the information bits combinasions
        - x: The polynom x^m
        
    # Returns:
        A list containing all the information bits polynoms multiplied by x^m
    """
    res = []
    
    for y in bits_info :
        res += [polynomial_multiplication(y,x)]
    
    return res
    
bits_info_multi = bits_info_multi_x_m(bits_info,poly_x_m(3))
print('Bits info multi :')
for i in range(len(bits_info_multi)):
    print(bits_info_multi[i])
print()


def compute_parity_bits(bits_info_multi, g):
    """ A function that computes the parity bits of the codewords
    
    # Arguments:
        - bits_info_multi: A list containing all the polynoms corresponding to the information bits multiplied by x^m
        - g: The generator polynom
        
    # Returns:
        A list containing all the parity bits corresponding to each codeword
    """
    res = []
    
    for i in bits_info_multi : 

        res += [rest_division(i,g)]
   
    return res

bits_parite = compute_parity_bits(bits_info_multi_x_m(bits_info,poly_x_m(3)),g)
print('Bits de parité :')
for i in range(len(bits_parite)):
    print(bits_parite[i])
print()


def systematic_form(bits_info, bits_parite, m):
    """ A function that computes the systematic form of the cyclic codewords
    
    # Arguments:
        - bits_info: A list containing all the information bits of the cyclic codewords
        - bits_parite: A list contaning all the parity bits of the cyclic codewords
        - m: The number of parity bits
    
    """
    res = []
    
    for i in range(len(bits_info)): 
        sys_form = np.zeros(len(bits_info[0])+m)
        sys_form = sys_form.tolist()
        sys_form[m:] = bits_info[i][:]
        sys_form[:m] = bits_parite[i][:m]
        
        res += [sys_form]
        
    return res


print('Forme systématique :')
sys_form = systematic_form(bits_info,bits_parite,m)
for i in range(len(sys_form)):
    print(sys_form[i])

[0, 1, 0, 0]
Bits info multi :
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 1, 0]
[0, 0, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 1]
[0, 0, 0, 1, 1, 0, 1]
[0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 1, 1, 1, 1]

Bits de parité :
[0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0]
[0, 1, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0]

Forme systématique :
[0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0]
[0, 1, 1, 0, 1, 0, 0]
[1, 0, 1, 1, 1, 0, 0]
[1, 1, 1, 0, 0, 1, 0]
[0, 0, 1, 1, 0, 1, 0]
[1, 0, 0, 0, 1, 1, 0]
[0, 1, 0, 1, 1, 1, 0]
[1, 0, 1, 0, 0, 0, 1]
[0, 1, 1, 1, 0, 0, 1]
[1, 1

## Simulation d'erreurs

Dans les questions précédentes, on a calculé le nombre d'erreurs corrigibles par ce code cyclique. Afin de savoir si le message reçu contient une erreur, on calcule le syndrome s(x) qui est le reste du polynôme correspondant à la forme systématique (r(x)) par le polynôme générateur (g(x)), pour résumer :  

s(x) = reste(r(x)/g(x))  

Si s(x) est égal à 0, cela signifie que le message est reçu sans erreur, en revanche s'il est différent de 0 cela veut dire que le message reçu comporte une erreur.    

Pour illustrer cet exemple, on va prendre le la forme systématique du message correspondant à 2, soit "*0 1 1 0 1 0 0*". Dans un premier temps on va simuler une erreur sur le bit d0, dans un deuxième temps deux erreurs, une sur le bit d0 et une autre sur le bit d1 et enfin trois erreurs une sur le bit d0, une sur le bit d1 et une sur le bit d3.

7) Calculez le syndrome pour le message sans erreurs, puis pour le message avec une seule erreur, pour celui comportant deux erreurs et enfin pour celui comportant 3 erreurs. Que peut-on en conclure?

In [76]:
print('Message sans erreur : ')
r = [0,1,1,0,1,0,0]
s_x = rest_division(r,g)
print(s_x)
print()


print('Message avec une erreur sur d0 :')
r = [1,1,1,0,1,0,0]
s_x = rest_division(r,g)
print(s_x)
print()

print('Message avec deux erreurs, une sur d0 et une sur d1 :')
r = [1,0,1,0,1,0,0]
s_x = rest_division(r,g)
print(s_x)
print()

print('Message avec trois erreurs, une sur d0, une sur d1  et une sur d3:')
r = [1,0,1,1,1,0,0]
s_x = rest_division(r,g)
print(s_x)
print()

Message sans erreur : 
[0, 0, 0, 0, 0, 0, 0]

Message avec une erreur sur d0 :
[1, 0, 0, 0, 0, 0, 0]

Message avec deux erreurs, une sur d0 et une sur d1 :
[1, 1, 0, 0, 0, 0, 0]

Message avec trois erreurs, une sur d0, une sur d1  et une sur d3:
[0, 0, 0, 0, 0, 0, 0]



Le syndrome reconnait correctement les erreurs. Malheureusement, le message à trois erreurs revient au message correspondant à 4, ce qui ne peut pas être détecté.

In [77]:
print('Message avec deux erreurs non à la suite :')
r = [1,1,0,0,1,0,0]
s_x = rest_division(r,g)
print(s_x)
print()

Message avec deux erreurs non à la suite :
[1, 0, 1, 0, 0, 0, 0]



Le détecteur reconnait deux erreurs qui ne sont pas à la suite ainsi que leur emplacement.